In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install transformers
!pip install transformers datasets evaluate
!pip install --upgrade accelerate
!pip install ray[tune]
!pip install accelerate -U

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
file_name = path + 'final_data.csv'

df = pd.read_csv(file_name)
df = df.drop(columns=['Unnamed: 0', 'id', 'Labels'])
df.head() ## The Text here is concatenation of Title and Post

,Text,AL,MA,PPE,CDU,TP,OTH
0,"Tryna get high on suboxone. Hello, I’ve got fo...",0,1,0,1,0,0
1,"Pre-employment drug test for hospital. Hello,\...",0,0,0,0,0,1
2,"Realllly ready to get off this stuff, jumped d...",0,0,1,0,1,0
3,Hey fellow suboxone peeps!. I get mine prescri...,0,0,1,1,0,0
4,Quickmd honesty?. On day 5 of bernese method.....,1,0,0,1,0,0


In [ ]:
# # No need to run this code again and again

# import pandas as pd
# from sklearn.model_selection import train_test_split


# train_val, test = train_test_split(df, test_size=0.1, random_state=42)

# # Splitting the remaining data into validation and test sets
# train, val = train_test_split(train_val, test_size=0.111, random_state=42)
# train = train.reset_index(drop=True)
# val = val.reset_index(drop=True)
# test = test.reset_index(drop=True)

# # Printing the sizes of the resulting sets
# print("Train set size:", len(train))
# print("Validation set size:", len(val))
# print("Test set size:", len(test))

# train.to_csv(path + 'train.csv', index = False)
# val.to_csv(path + 'val.csv', index = False)
# test.to_csv(path + 'test.csv', index = False)

Train set size: 4066
Validation set size: 508
Test set size: 509


## Reading training, validation and test set samples

# BackBone Code

In [ ]:
%%capture
!pip install transformers
!pip install transformers datasets evaluate
!pip install --upgrade accelerate
!pip install ray[tune]
!pip install accelerate -U

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
train = pd.read_csv(path + 'train.csv')
val = pd.read_csv(path + 'val.csv')
test = pd.read_csv(path + 'test.csv')
train

,Text,AL,MA,PPE,CDU,TP,OTH
0,To tell Or not to tell. Do you disclose to you...,0,0,0,0,0,1
1,Please need honest experience Please respond!!...,0,1,0,1,0,0
2,kratom and suboxone?. Is fhere a benefit to ta...,0,0,0,1,0,0
3,"Extraction. As I’m sure many of you know, subu...",0,1,0,1,0,0
4,Question on my prescription of suboxone?. So i...,0,1,0,0,0,0
...,...,...,...,...,...,...,...
4061,Down to 2mg a day. Finally successfully lowere...,0,0,0,1,1,0
4062,"Close To The Jump: What To Expect?. Hi all,\n\...",0,0,0,0,1,0
4063,Will smoking Salvia (Salvinorin A - high effic...,0,0,0,1,0,0
4064,Looking to get on Sub or Methadone Program - A...,1,0,0,0,0,0


In [ ]:
import datasets
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)
test = Dataset.from_pandas(test)


dataset = DatasetDict()

dataset['train'] = train
dataset['val'] = val
dataset['test'] = test

dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 4066
    })
    val: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 508
    })
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})

In [ ]:
example = dataset['train'][0]
example

{'Text': "To tell Or not to tell. Do you disclose to your doctors that you are an addict ?  Why ?  Has it ever backfired per se ? Do you think the treatment you received was the same as the non addict ?\n\n\n  Once I found it necessary (who knows why ?) , but I no longer share that I'm an addict. I take 1mg of subs and don't see any drug interactions that can occur. If by any miniscule chance its a possibility- I assume the risk. I will never disclose to doctors of my prior drug use. My last drink was in 2008,,,,, what good comes out of telling an orthopedic surgeon  I'm an alcoholic ?  None,  ime it affected treatment- to the negative.  \nImo, there's little to no relevance,,, that coupled by the high probability that it will adversely affect treatment- i say Dont Tell.",
 'AL': 0,
 'MA': 0,
 'PPE': 0,
 'CDU': 0,
 'TP': 0,
 'OTH': 1}

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


In [ ]:
%%time
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime

timestamp = datetime.now().strftime('%Y-%m-%d')
model_name = "microsoft/mpnet-base"
model_checkpoint = path + model_name + "-"+timestamp

print(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

/content/drive/MyDrive/Omar/AAAI-Submission/microsoft/mpnet-base-2023-08-08


CPU times: user 271 ms, sys: 16 ms, total: 287 ms
Wall time: 1.76 s


In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
example = encoded_dataset['train'][0]
print(example.keys())

Map:   0%|          | 0/4066 [00:00<?, ? examples/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
print([id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0])

<s> to tell or not to tell. do you disclose to your doctors that you are an addict? why? has it ever backfired per se? do you think the treatment you received was the same as the non addict? once i found it necessary ( who knows why? ), but i no longer share that i'm an addict. i take 1mg of subs and don't see any drug interactions that can occur. if by any miniscule chance its a possibility - i assume the risk. i will never disclose to doctors of my prior drug use. my last drink was in 2008,,,,, what good comes out of telling an orthopedic surgeon i'm an alcoholic? none, ime it affected treatment - to the negative. imo, there's little to no relevance,,, that coupled by the high probability that it will adversely affect treatment - i say dont tell. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from ast import arg
batch_size = 16

metric_name = 'f1'

from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    f"Misc", ## Saving the values in a check-point
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'weighted')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_weighted_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
#Just for checking
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7113, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1080, -0.0171, -0.0135,  0.1159,  0.0468,  0.0301]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
%%time
trainer.train() ## Training the model

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.429006,0.575620,0.708603,0.301181
2,0.448700,0.356760,0.670428,0.772889,0.401575
3,0.448700,0.324943,0.704283,0.790144,0.446850
4,0.286500,0.289925,0.771981,0.834678,0.531496
5,0.286500,0.286192,0.774773,0.836976,0.527559
6,0.210000,0.280860,0.770265,0.833531,0.523622
7,0.210000,0.287537,0.764013,0.834022,0.513780
8,0.162600,0.284866,0.779994,0.843150,0.535433
9,0.162600,0.286548,0.779424,0.842444,0.523622
10,0.134300,0.287248,0.786757,0.849113,0.537402


CPU times: user 22min 37s, sys: 16.2 s, total: 22min 53s
Wall time: 22min 11s


TrainOutput(global_step=2550, training_loss=0.24615600511139515, metrics={'train_runtime': 1331.1404, 'train_samples_per_second': 30.545, 'train_steps_per_second': 1.916, 'total_flos': 1.069847972646912e+16, 'train_loss': 0.24615600511139515, 'epoch': 10.0})

In [ ]:
## Result on the evaluation set
trainer.evaluate()

{'eval_loss': 0.2872484624385834,
 'eval_f1': 0.7867573084397264,
 'eval_roc_auc': 0.8491132663673338,
 'eval_accuracy': 0.5374015748031497,
 'eval_runtime': 5.3455,
 'eval_samples_per_second': 95.033,
 'eval_steps_per_second': 5.986,
 'epoch': 10.0}

In [ ]:
#Saving the model
trainer.save_model(model_checkpoint)

# Trained model Loading and Prediction

In [ ]:
%%capture
!pip install transformers
!pip install transformers datasets evaluate
!pip install --upgrade accelerate
!pip install ray[tune]
!pip install accelerate -U

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split

## BERT-BASE model

In [ ]:
%%time
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

from transformers import AutoTokenizer
import numpy as np
from datetime import datetime

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
CPU times: user 1.2 s, sys: 59.5 ms, total: 1.26 s
Wall time: 5.3 s


In [ ]:
def saving_predictions(lst, mdl_nm):
  labelss = ['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
  xx = []
  for ls in lst:
      xy = []
      for i in range(len(ls)):
          if(ls[i]==True):
            xy.append(labelss[i])
      xx.append(xy)
  print(len(xx), xx)
  df = pd.DataFrame({'Predictions': xx})
  pathx = '/content/drive/MyDrive/Omar/AAAI-Submission/ChatGPT-Exps/'
  df.to_csv(pathx+f'{mdl_nm}-Predictions.csv', index= False)


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

model_name = "bert-base-uncased"
## Check te checkpoint carefully
model_checkpoint = "/content/drive/MyDrive/Omar/AAAI-Submission/bert-base-uncased-2023-07-18"

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'Bert')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])

509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False False]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]]
509 [['MA', 'PPE'], ['PPE', 'TP'], [], [], [], ['PPE', 'TP'], ['CDU', 'TP'], [], ['AL'], ['CDU'], ['AL'], ['PPE'], ['MA', 'TP'], ['PPE'], ['MA'], ['TP'], ['OTH'], ['PPE'], ['PPE'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['TP'], ['MA', 'TP'], ['MA'], ['PPE'], ['AL'], ['PPE', 'TP'], ['CDU'], ['OTH'], [], [], ['PPE', 'TP'], [], ['AL'], ['AL'], ['AL'], ['PPE', 'TP'], ['AL'], ['CDU'], ['CDU'], ['AL'], ['CDU'], ['PPE', 'TP'], ['PPE'], ['MA'], ['MA', 'TP'], ['AL'], [], ['PPE', 'TP'], ['PPE', 'TP'], ['

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## ALBERT Model

In [ ]:
%%time
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

from transformers import AutoTokenizer
import numpy as np
from datetime import datetime

model_name = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
CPU times: user 751 ms, sys: 30.5 ms, total: 781 ms
Wall time: 3.46 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

model_name = "albert-base-v2"
## Check te checkpoint carefully
model_checkpoint = "/content/drive/MyDrive/Omar/AAAI-Submission/albert-base-v2-2023-07-18"

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'ALBert')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])

509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True False False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False False False False]]
509 [['MA'], ['PPE', 'TP'], ['OTH'], [], [], ['PPE', 'TP'], ['CDU'], ['AL'], ['AL'], ['CDU'], [], [], ['TP'], ['PPE'], ['MA'], ['TP'], ['OTH'], ['MA', 'CDU'], ['PPE'], ['PPE', 'TP'], ['MA', 'CDU'], ['TP'], ['TP'], ['MA', 'CDU'], ['AL'], ['PPE'], ['AL'], ['PPE', 'TP'], ['MA', 'CDU'], [], [], ['OTH'], ['PPE', 'TP'], ['OTH'], ['AL'], ['AL'], ['AL'], ['TP'], ['AL'], ['MA', 'CDU'], ['CDU'], ['AL'], ['CDU'], ['TP'], ['PPE'], ['MA'], ['MA', 'TP'], ['AL'], ['AL'], ['PPE', 'TP'], ['PPE', 'CDU', 'TP'], ['AL'], ['MA']

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## ROBERTa-Base Model

In [ ]:
%%time

##Loading the dataset
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

## Endcoding the model according to data
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

## Encoding the Dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])
CPU times: user 725 ms, sys: 18.9 ms, total: 744 ms
Wall time: 3.61 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

model_name = "roberta-base"
## Check te checkpoint carefully
model_checkpoint = "/content/drive/MyDrive/Omar/AAAI-Submission/roberta-base-2023-07-18"

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

## Previously trained model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'RoBERTa')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])


509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True False False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True  True  True False False]]
509 [['MA'], ['PPE', 'TP'], ['OTH'], ['TP'], ['TP'], ['PPE', 'TP'], ['CDU', 'TP'], ['AL'], ['AL'], ['MA', 'PPE', 'CDU'], ['MA', 'PPE'], ['OTH'], ['TP'], ['MA', 'PPE'], ['MA'], ['TP'], [], ['PPE', 'CDU'], ['PPE'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['TP'], ['TP'], ['MA', 'CDU', 'TP'], ['AL'], ['PPE'], ['AL'], ['TP'], ['MA', 'PPE', 'CDU'], [], [], ['PPE'], ['PPE', 'TP'], [], ['AL'], ['AL'], ['AL'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['CDU'], ['AL'], ['PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['PPE'], ['MA', 'P

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Distil-BERT model

In [ ]:
%%time

##Loading the dataset
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

## Endcoding the model according to data
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

## Encoding the Dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])
CPU times: user 635 ms, sys: 22.9 ms, total: 658 ms
Wall time: 2.36 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

## Check te checkpoint carefully
model_checkpoint = f"/content/drive/MyDrive/Omar/AAAI-Submission/{model_name}-2023-07-18"
print(model_checkpoint)

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

## Previously trained model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'Distil-BERT')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])


/content/drive/MyDrive/Omar/AAAI-Submission/distilbert-base-uncased-2023-07-18


509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True False False False False]
 [False False False False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]]
509 [['MA'], ['TP'], ['OTH'], [], [], ['PPE', 'TP'], ['CDU', 'TP'], ['AL'], ['AL'], ['MA', 'PPE', 'CDU'], ['AL'], ['PPE'], ['MA', 'TP'], ['PPE'], ['MA'], ['CDU', 'TP'], ['OTH'], ['MA'], ['PPE'], ['TP'], ['MA', 'PPE', 'CDU'], ['TP'], ['TP'], ['CDU', 'TP'], ['MA'], ['PPE'], [], ['PPE', 'TP'], ['MA', 'CDU'], ['OTH'], [], [], ['PPE', 'TP'], [], ['AL'], ['AL'], ['AL'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['CDU'], ['AL'], ['MA', 'CDU'], ['TP'], ['PPE'], ['MA'], ['TP'], ['AL'], ['MA'], ['PPE', 'TP'], ['PPE'], ['

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## ELECTRA Model

In [ ]:
%%time

##Loading the dataset
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

## Endcoding the model according to data
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
model_name = "google/electra-base-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

## Encoding the Dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
CPU times: user 715 ms, sys: 16.3 ms, total: 731 ms
Wall time: 3.53 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

## Check te checkpoint carefully
model_checkpoint = f"/content/drive/MyDrive/Omar/AAAI-Submission/{model_name}-2023-07-18"
print(model_checkpoint)

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

## Previously trained model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'ELECTRA')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])


/content/drive/MyDrive/Omar/AAAI-Submission/google/electra-base-discriminator-2023-07-18


509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True False  True False False]
 [False False  True False  True False]
 [False False False False False False]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True  True  True False False]]
509 [['MA', 'CDU'], ['PPE', 'TP'], [], ['PPE'], ['OTH'], ['PPE', 'TP'], ['CDU', 'TP'], ['AL'], ['AL'], ['MA', 'PPE', 'CDU'], ['PPE'], [], ['MA', 'TP'], ['MA', 'PPE'], ['MA'], ['CDU', 'TP'], ['OTH'], ['PPE', 'CDU'], ['PPE'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['TP'], ['CDU', 'TP'], ['AL'], ['PPE'], ['AL'], ['PPE', 'TP'], ['CDU'], ['TP'], ['MA'], [], ['PPE', 'TP'], ['OTH'], ['AL'], ['AL'], ['AL'], ['TP'], ['AL'], ['CDU'], ['CDU'], ['AL'], ['PPE', 'CDU', 'TP'], ['PPE', 'CDU', 'TP'], ['PPE'], [

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## XLNet Model

In [ ]:
%%time

##Loading the dataset
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

## Endcoding the model according to data
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
model_name = "xlnet-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

## Encoding the Dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
CPU times: user 698 ms, sys: 6.72 ms, total: 704 ms
Wall time: 1.21 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

## Check te checkpoint carefully
model_checkpoint = f"/content/drive/MyDrive/Omar/AAAI-Submission/{model_name}-2023-07-18"
print(model_checkpoint)

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

## Previously trained model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'XLNet')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])


/content/drive/MyDrive/Omar/AAAI-Submission/xlnet-base-cased-2023-07-18


509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False  True False False False False]
 [False False  True False  True False]
 [False False False False False False]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]]
509 [['MA'], ['PPE', 'TP'], [], [], ['OTH'], ['PPE', 'TP'], ['PPE', 'CDU', 'TP'], ['AL'], ['AL'], ['CDU'], ['MA'], [], ['PPE', 'TP'], ['MA', 'PPE'], ['MA'], ['TP'], ['OTH'], ['PPE', 'CDU'], ['PPE'], ['TP'], ['PPE', 'CDU'], ['MA', 'TP'], ['CDU', 'TP'], ['CDU', 'TP'], ['AL'], ['PPE'], ['AL'], ['PPE', 'TP'], ['MA', 'CDU'], ['TP'], ['PPE'], ['OTH'], ['PPE', 'TP'], ['MA'], ['AL'], ['AL'], ['AL', 'MA', 'PPE'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['CDU'], ['AL'], ['PPE', 'CDU', 'TP'], ['PPE', 'CDU', 'TP'], ['PPE

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## MPNet-Base

In [ ]:
%%time

##Loading the dataset
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
test = pd.read_csv(path + 'test.csv')

import datasets
from datasets import Dataset, DatasetDict
test = Dataset.from_pandas(test)

dataset = DatasetDict()
dataset['test'] = test

print(dataset)

labels = [label for label in dataset['test'].features.keys() if label not in ['Text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

print(labels)
print(id2label)

## Endcoding the model according to data
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
model_name = "microsoft/mpnet-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

## Encoding the Dataset
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)
example = encoded_dataset['test'][0]
print(example.keys())

encoded_dataset.set_format("torch")

DatasetDict({
    test: Dataset({
        features: ['Text', 'AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH'],
        num_rows: 509
    })
})
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
{0: 'AL', 1: 'MA', 2: 'PPE', 3: 'CDU', 4: 'TP', 5: 'OTH'}


Map:   0%|          | 0/509 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])
CPU times: user 653 ms, sys: 14.7 ms, total: 667 ms
Wall time: 2.7 s


In [ ]:
## Loading previously trained model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

## Check te checkpoint carefully
model_checkpoint = f"/content/drive/MyDrive/Omar/AAAI-Submission/{model_name}-2023-08-08"
print(model_checkpoint)

modelX = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, local_files_only=True)
trainerX = Trainer(model=modelX)
trainerX.model = modelX.cuda()

## Previously trained model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, hamming_loss

def compute_metrics2(pred, labels):
    # Process the predictions and compute evaluation metrics
    # Assuming the labels are in multihot format
    # Convert the multihot encoded labels to binary
    binary_labels = (pred.label_ids > 0.5)
    binary_predictions = (pred.predictions > 0.5)
    print(len(binary_labels), len(binary_predictions))
    print(binary_labels, binary_predictions)
    saving_predictions(binary_predictions, 'MPNet')
    return {
        'classification_report': classification_report(binary_labels, binary_predictions, target_names=labels),
        'accuracy': accuracy_score(binary_labels, binary_predictions),
        'precision': precision_score(binary_labels, binary_predictions, average='weighted'),
        'recall': recall_score(binary_labels, binary_predictions, average='weighted'),
        'f1': f1_score(binary_labels, binary_predictions, average='weighted'),
        'hamming_loss': hamming_loss(binary_labels, binary_predictions)
    }

##Get the raw probabilites in the test set
tg = trainerX.predict(encoded_dataset['test'])
tg

tx= compute_metrics2(tg, labels)
print(tx['classification_report'])
print(tx['precision'])
print(tx['recall'])
print(tx['f1'])


/content/drive/MyDrive/Omar/AAAI-Submission/microsoft/mpnet-base-2023-08-08


509 509
[[False  True  True False False False]
 [False False  True False  True False]
 [False False False False False  True]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]] [[False False  True  True False False]
 [False False  True False  True False]
 [False False False  True False False]
 ...
 [False False False  True False False]
 [False  True False False False False]
 [False  True False  True False False]]
509 [['PPE', 'CDU'], ['PPE', 'TP'], ['CDU'], [], [], ['PPE', 'TP'], ['CDU', 'TP'], ['AL'], ['AL'], ['CDU'], [], [], ['MA', 'TP'], [], ['MA'], ['TP'], ['OTH'], ['PPE', 'CDU'], ['PPE'], ['TP'], ['MA', 'PPE', 'CDU'], ['TP'], ['TP'], ['CDU', 'TP'], ['AL', 'CDU'], ['PPE'], ['AL'], ['PPE', 'TP'], ['CDU'], ['TP'], [], [], ['PPE', 'TP'], ['OTH'], ['AL'], ['AL'], ['AL', 'MA'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['CDU'], ['AL'], ['PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['PPE'], ['MA', 'CDU'], ['MA', 'TP'], ['AL'], ['P

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
